#### Data 620 - Project 1 <br>June 19, 2019<br>Team 2: <ul> <li>Anthony Munoz</li> <li>Katie Evers</li> <li>Juliann McEachern</li> <li>Mia Siracusa</li></ul>

<h1 align="center">"Network Analysis"</h1>

For your first project, you are asked to:
1.  Identify and load a network dataset that has some categorical information available for each node.
2.  For each of the nodes in the dataset, calculate degree centrality and eigenvector centrality.
3.  Compare your centrality measures across your categorical groups.

For example, using the [dataset](http://researchnews.osu.edu/archive/chains.htm) from a study of romantic relationships among high school students in Columbus, Ohio , you might want to determine if there are any differences in your calculated centrality measures between the two sexes.  You might use a t-test or another statistical measure to support your findings.
You may work in a small group.  Your project should be delivered in an IPython Notebook, and posted in GitHub.  

## Network Dataset Selection

We choose to analyze NYC OpenData's [Women Resources Network Database](https://data.cityofnewyork.us/Social-Services/NYC-Women-s-Resource-Network-Database/pqg4-dm6b). The dataset includes 59 variables by columns and 1,147 rows as of the most recent web update on Nov 7, 2018.

Our focus will be on the identified organizations that serve New York City women and families. We will also be examining this network relationships by **insert variable name here**. Each node contains categorical information on which borough the organization services.



In [5]:
#importing the libraries 
import networkx as nx
import pandas as pd
import numpy as np

# to import the socrata library you have to go to Anaconda terminal and install the package before importing 
# the library can be install using "pip install sodapy"
from sodapy import Socrata
import warnings 

#warnings.filterwarnings('ignore') UNHASH AFTER COMPLETION

NYC OpenData was accessed using Socrata API. The Women Resources Network Database then was obtained from the client connection below and stored in a pandas dataframe. 

In [26]:
client = Socrata("data.cityofnewyork.us",None)
results = client.get("pqg4-dm6b", limit=2000)
df = pd.DataFrame(results)

In [27]:
#checking the dataframe information 
df.info(verbose= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1147 entries, 0 to 1146
Data columns (total 57 columns):
address1                                            1074 non-null object
address2                                            145 non-null object
aging                                               1147 non-null object
anti_discrimination_human_rights                    1147 non-null object
arts_culture                                        1147 non-null object
bbl                                                 758 non-null object
bin                                                 758 non-null object
borough                                             771 non-null object
bronx                                               1147 non-null object
brooklyn                                            1147 non-null object
business                                            1147 non-null object
census_tract                                        771 non-null object
child_care_parent_info

In [29]:
#how to subset dataframe
df_subset = df[["organizationname","census_tract","manhattan","queens","staten_island","brooklyn","bronx","faith_based_organization"]]

,organizationname,borough,faith_based_organization
0,YWCA of the City of New York,manhattan,Y
1,RightRides for Women's Safety,manhattan,Y
2,Charles B. Wang Community Health Center,manhattan,Y
3,American-Italian Cancer Foundation's Mobile Ma...,manhattan,Y
4,QHC-Diabetes Center,manhattan,N
5,Legal Momentum,manhattan,Y
6,Young Mothers Program/BBBS,manhattan,Y
7,Shorefront YM-YWHA,manhattan,N
8,Federally Employed Women (FEW),manhattan,Y
9,Sauti Yetu Center for African,manhattan,Y


In [31]:
#reshape data to long dataframe to build nodes / edges
reshape = df_subset.melt(id_vars='organizationname', value_vars=list(['manhattan','queens','brooklyn','bronx','staten_island']), var_name='borough', value_name='faith_based_organization')

edges = df[["organizationname","borough"]]
edges

,organizationname,borough
0,YWCA of the City of New York,NaN
1,RightRides for Women's Safety,BROOKLYN
2,Charles B. Wang Community Health Center,NaN
3,American-Italian Cancer Foundation's Mobile Ma...,MANHATTAN
4,QHC-Diabetes Center,QUEENS
5,Legal Momentum,MANHATTAN
6,Young Mothers Program/BBBS,MANHATTAN
7,Shorefront YM-YWHA,BROOKLYN
8,Federally Employed Women (FEW),NaN
9,Sauti Yetu Center for African,NaN
